In [25]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import sys
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap, TyreRaceData, Team, DriverTeamSession, TeamCircuitStats

import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from itertools import product

# Initialize database connection
global db_session
engine, db_session = init_db()


def all_drivers_tyre(year, round):
	# Query to get the last 30 race sessions globally prior to the specified year/round
	global_past_races = (
		db_session.query(Session.session_id)
		.join(RacingWeekend, RacingWeekend.racing_weekend_id == Session.weekend_id)
		.filter(
			Session.session_type == "Race",
			# Filter races strictly before the specified year/round
			(RacingWeekend.year < year) |
			((RacingWeekend.year == year) & (RacingWeekend.round < round))
		)
		.order_by(RacingWeekend.year.desc(), RacingWeekend.round.desc())  # Order by most recent first
		.limit(30)  # Limit to the last 30 races
		.all()
	)

	# Extract session IDs from the query result
	global_session_ids = [race.session_id for race in global_past_races]

	# Query tyre data for ALL drivers across the last 30 races globally
	global_tyre_data = (
		db_session.query(TyreRaceData.tyre_type, TyreRaceData.a, TyreRaceData.b, TyreRaceData.c)
		.filter(TyreRaceData.race_id.in_(global_session_ids))
		.all()
	)

	# Group and calculate global averages
	global_tyre_stats = {}
	for tyre_type, a, b, c in global_tyre_data:
		if tyre_type not in global_tyre_stats:
			global_tyre_stats[tyre_type] = {"a": [], "b": [], "c": []}

		global_tyre_stats[tyre_type]["a"].append(a)
		global_tyre_stats[tyre_type]["b"].append(b)
		global_tyre_stats[tyre_type]["c"].append(c)

	# Calculate the global averages for each tyre type
	global_averaged_tyre_stats = {}
	for tyre_type, stats in global_tyre_stats.items():
		avg_a = sum(stats["a"]) / len(stats["a"]) if stats["a"] else 0
		avg_b = sum(stats["b"]) / len(stats["b"]) if stats["b"] else 0
		avg_c = sum(stats["c"]) / len(stats["c"]) if stats["c"] else 0

		global_averaged_tyre_stats[tyre_type] = {
			"avg_a": avg_a,
			"avg_b": avg_b,
			"avg_c": avg_c,
		}
	
	return global_averaged_tyre_stats

def get_starting_grid(session_id):
	session_results = db_session.query(SessionResult).filter_by(session_id=session_id).all()

	starting_grid = {}
	for pos in session_results:
		starting_grid[pos.position] = pos.driver_id

	return starting_grid

def get_laps(session_id):
	max_lap = db_session.query(func.max(Lap.lap_num)).filter(Lap.session_id == session_id).scalar()

	# If no laps are found, return 0
	return max_lap if max_lap is not None else 0

def get_all_data(year, round):
	# get all drivers that competed
	racing_weekend = db_session.query(RacingWeekend).filter_by(year=year, round=round).first()

	quali_session = db_session.query(Session).filter_by(weekend_id=racing_weekend.racing_weekend_id, session_type="Qualifying").first()
	race_session = db_session.query(Session).filter_by(weekend_id=racing_weekend.racing_weekend_id, session_type="Race").first()

	# find drivers
	drivers = db_session.query(DriverTeamSession).filter_by(session_id=race_session.session_id).all()

	all_driver_tyre_deg = all_drivers_tyre(year, round)
	driver_tyre_deg = {}
	for driver_entry in drivers:
		driver = driver_entry.driver

		# Query to get the last 30 race sessions the driver participated in
		past_races = (
			db_session.query(Session.session_id)
			.join(DriverTeamSession, DriverTeamSession.session_id == Session.session_id)
			.join(RacingWeekend, RacingWeekend.racing_weekend_id == Session.weekend_id)
			.filter(
				DriverTeamSession.driver_id == driver.driver_id,
				Session.session_type == "Race",
				# Filter races strictly before the specified year/round
				(RacingWeekend.year < year) |
				((RacingWeekend.year == year) & (RacingWeekend.round < round))
			)
			.order_by(RacingWeekend.year.desc(), RacingWeekend.round.desc())  # Order by most recent first
			.limit(30)  # Limit to the last 30 races
			.all()
		)

		# for race in past_races:
		# 	print(race.round)
		# print("\n\n\n")

		if len(past_races) < 20:
			driver_tyre_deg[driver.driver_id] = all_driver_tyre_deg
			continue

		session_ids = [race.session_id for race in past_races]

		# Get all tyre data for those sessions and driver
		tyre_data = db_session.query(TyreRaceData.tyre_type, TyreRaceData.a, TyreRaceData.b, TyreRaceData.c)\
			.filter(TyreRaceData.driver_id == driver.driver_id,
					TyreRaceData.race_id.in_(session_ids))\
			.all()
		
		# print(len(tyre_data))
		tyre_stats = {}

		# Loop over the queried tyre_data
		for tyre_type, a, b, c in tyre_data:
			if tyre_type == 4:
				continue
			if tyre_type not in tyre_stats:
				tyre_stats[tyre_type] = {"a": [], "b": [], "c": []}
			
			# Append the values of a, b, and c for this tyre type
			tyre_stats[tyre_type]["a"].append(a)
			tyre_stats[tyre_type]["b"].append(b)
			tyre_stats[tyre_type]["c"].append(c)

		# Calculate the averages for each tyre type
		averaged_tyre_stats = {}
		for tyre_type, stats in tyre_stats.items():
			avg_a = sum(stats["a"]) / len(stats["a"]) if stats["a"] else 0
			avg_b = sum(stats["b"]) / len(stats["b"]) if stats["b"] else 0
			avg_c = sum(stats["c"]) / len(stats["c"]) if stats["c"] else 0
			
			averaged_tyre_stats[tyre_type] = {
				"avg_a": avg_a,
				"avg_b": avg_b,
				"avg_c": avg_c,
			}

		driver_tyre_deg[driver.driver_id] = averaged_tyre_stats

	starting_grid = get_starting_grid(quali_session.session_id)
	
	num_laps = get_laps(race_session.session_id)

	return starting_grid, driver_tyre_deg, num_laps


starting_grid, driver_tyre_deg, num_laps = get_all_data(2024,18)

Exhaustive search

In [27]:
def simulate_pit_stops(driver_tyre_deg, num_laps, pit_time=20):
    optimal_pit_stops = {}
    for driver_id, tyre_data in driver_tyre_deg.items():
        possible_tyres = list(tyre_data.keys())
        if len(possible_tyres) < 2:
            # Not enough tyre types to change; skip or handle accordingly
            optimal_pit_stops[driver_id] = {
                'pit_lap': None,
                'start_tyre': None,
                'new_tyre': None,
                'total_time': float('inf')
            }
            continue
        
        # Precompute cumulative lap times for each tyre
        cumulative = {}
        for tyre in possible_tyres:
            a = tyre_data[tyre]['avg_a']
            b = tyre_data[tyre]['avg_b']
            c = tyre_data[tyre]['avg_c']
            cumulative[tyre] = [0.0]  # index 0 is 0 laps
            total_time = 0.0
            for x in range(1, num_laps + 1):
                lap_time = a * (x ** 2) + b * x + c
                total_time += lap_time
                cumulative[tyre].append(total_time)

        # Find the optimal pit strategy
        min_total = float('inf')
        best_pit_lap = None
        best_start_tyre = None
        best_new_tyre = None
        
        for start_tyre in possible_tyres:
            for new_tyre in possible_tyres:
                if start_tyre == new_tyre:
                    continue
                for pit_lap in range(1, num_laps):
                    stint1_length = pit_lap
                    stint2_length = num_laps - pit_lap
                    if stint2_length < 1:
                        continue
                    # Ensure we don't exceed precomputed values
                    if stint1_length > len(cumulative[start_tyre]) - 1:
                        continue
                    if stint2_length > len(cumulative[new_tyre]) - 1:
                        continue
                    total = (cumulative[start_tyre][stint1_length] +
                             cumulative[new_tyre][stint2_length] +
                             pit_time)
                    if total < min_total:
                        min_total = total
                        best_pit_lap = pit_lap
                        best_start_tyre = start_tyre
                        best_new_tyre = new_tyre
        
        optimal_pit_stops[driver_id] = {
            'pit_lap': best_pit_lap,
            'start_tyre': best_start_tyre,
            'new_tyre': best_new_tyre,
            'total_time': min_total
        }
    
    return optimal_pit_stops

# Example usage:
starting_grid, driver_tyre_deg, num_laps = get_all_data(2024, 4)
optimal_pits = simulate_pit_stops(driver_tyre_deg, num_laps)

# Convert the optimal_pits dictionary to a DataFrame
df_optimal_pits = pd.DataFrame.from_dict(
    optimal_pits, 
    orient='index', 
    columns=['pit_lap', 'start_tyre', 'new_tyre', 'total_time']
).reset_index().rename(columns={'index': 'driver_id'})

(df_optimal_pits)

,driver_id,pit_lap,start_tyre,new_tyre,total_time
0,25,18,2,3,86.818672
1,1,16,2,3,84.730869
2,32,30,3,2,84.313877
3,4,19,1,3,83.037317
4,14,18,2,3,88.266285
5,3,19,2,3,86.759574
6,8,20,2,3,90.122897
7,5,23,2,3,93.573439
8,17,1,3,1,-38.934370
9,35,34,3,2,104.869966


Now need to look at:
Using cumaltive method within RF properly
Imporve cumaltive method - currently problem of on 2nd stint the tyre has already done laps
